In [1]:
import pandas as pd
import json
from sqlalchemy import create_engine
from ling_features import fog

conn_string = 'postgresql://10.101.13.99/crsp'

def expand_json(df, col):
    return pd.concat([df.drop([col], axis=1),
                      df[col].map(lambda x: json.loads(json.dumps(x))).apply(pd.Series)], axis=1)

engine = create_engine(conn_string)

sql =  """
SELECT file_name, last_update, speaker_number, context, section,
   speaker_text
FROM streetevents.speaker_data
WHERE file_name = '1000000_T' 
ORDER BY section, context, speaker_number"""

speaker_data = pd.read_sql(sql, engine)

In [2]:
speaker_data['last_update'] = speaker_data['last_update'].map(lambda x: str(x.astimezone('UTC')))
speaker_data['fog_data'] = speaker_data['speaker_text'].apply(fog)
speaker_data = expand_json(speaker_data, 'fog_data')
speaker_data = speaker_data.drop(['speaker_text'], axis=1)

In [3]:
speaker_data

,file_name,last_update,speaker_number,context,section,fog,complex_words,fog_words,fog_sents
0,1000000_T,2005-07-29 19:37:34+00:00,1,pres,1,10.000000,9.0,60.0,6.0
1,1000000_T,2005-07-29 19:37:34+00:00,2,pres,1,14.431818,51.0,330.0,16.0
2,1000000_T,2005-07-29 19:37:34+00:00,3,pres,1,13.739500,135.0,846.0,46.0
3,1000000_T,2005-07-29 19:37:34+00:00,4,pres,1,14.986884,115.0,670.0,33.0
4,1000000_T,2005-07-29 19:37:34+00:00,5,pres,1,14.805940,137.0,734.0,40.0
...,...,...,...,...,...,...,...,...,...
78,1000000_T,2005-07-29 19:37:34+00:00,74,qa,1,0.700000,0.0,7.0,4.0
79,1000000_T,2005-07-29 19:37:34+00:00,75,qa,1,1.600000,0.0,4.0,1.0
80,1000000_T,2005-07-29 19:37:34+00:00,76,qa,1,2.400000,0.0,6.0,1.0
81,1000000_T,2005-07-29 19:37:34+00:00,77,qa,1,8.133333,1.0,12.0,1.0


In [4]:
sql =  """
SELECT *
FROM se_features.fog_measure
WHERE file_name = '1000000_T' 
ORDER BY section, context, speaker_number"""

orig_data = pd.read_sql(sql, engine)

In [5]:
orig_data

,file_name,last_update,speaker_number,context,section,fog,complex_words,fog_words,fog_sents
0,1000000_T,2005-07-29 19:37:34+00:00,1,pres,1,10.000000,9.0,60.0,6.0
1,1000000_T,2005-07-29 19:37:34+00:00,2,pres,1,14.431818,51.0,330.0,16.0
2,1000000_T,2005-07-29 19:37:34+00:00,3,pres,1,13.739500,135.0,846.0,46.0
3,1000000_T,2005-07-29 19:37:34+00:00,4,pres,1,14.986884,115.0,670.0,33.0
4,1000000_T,2005-07-29 19:37:34+00:00,5,pres,1,14.805940,137.0,734.0,40.0
...,...,...,...,...,...,...,...,...,...
78,1000000_T,2005-07-29 19:37:34+00:00,74,qa,1,0.700000,0.0,7.0,4.0
79,1000000_T,2005-07-29 19:37:34+00:00,75,qa,1,1.600000,0.0,4.0,1.0
80,1000000_T,2005-07-29 19:37:34+00:00,76,qa,1,2.400000,0.0,6.0,1.0
81,1000000_T,2005-07-29 19:37:34+00:00,77,qa,1,8.133333,1.0,12.0,1.0
